### 20/11/2024

In [1]:
experiment_name = "k_fold_enet_bce"
epochs = 100
data_path = "/home/sebastian/codes/data/ABGQI_mel_spectrograms"
early_stopping = 10
batch_size = 64
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Use GPU 1

In [2]:
import os
import argparse
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np
import time
from tqdm import tqdm
from sklearn.model_selection import KFold
import csv
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
def bytes_to_mb(memory_bytes):
    """
    Takes the memory usage in bytes as input and returns the memory usage converted to megabytes (MB).
    """
    return memory_bytes / (1024 * 1024)
def model_memory_usage_alternative(model, input_dtype=torch.float32):
    total_params = sum(p.numel() for p in model.parameters())
    dtype_size = torch.tensor([], dtype=input_dtype).element_size()  # Size of the data type
    total_memory = total_params * dtype_size  # Total memory in bytes
    return total_memory / (1024 ** 2)

In [5]:
def save_fold_metrics(metrics, fold_summary, filename):
    # Comprobar si el archivo ya existe
    if os.path.exists(filename):
        # Leer el archivo existente
        existing_metrics = pd.read_csv(filename)
        # Concatenar las métricas actuales con las existentes
        new_data = pd.DataFrame([fold_summary])
        updated_metrics = pd.concat([existing_metrics, new_data], ignore_index=True)
    else:
        # Si no existe, solo usar las nuevas métricas
        updated_metrics = pd.DataFrame([fold_summary])

    # Guardar las métricas actualizadas
    updated_metrics.to_csv(filename, index=False)

In [6]:
class EarlyStopping:
    def __init__(self, patience=5):
        self.patience = patience
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

In [7]:
def get_args_parser():
    parser = argparse.ArgumentParser(description="Train and test a model with folder-based data structure using K-Fold Cross-Validation.")
    parser.add_argument('--experiment_name', type=str, default=experiment_name)
    parser.add_argument('--im_size', type=tuple, default=(300, 300))
    parser.add_argument('--batch_size', type=int, default=batch_size)
    parser.add_argument('--epochs', type=int, default=epochs)
    parser.add_argument('--data_path', type=str, default=data_path)
    parser.add_argument('--seed', type=int, default=0)
    parser.add_argument('--early_stopping', type=int, default=early_stopping)
    parser.add_argument('--k_folds', type=int, default=5)
    return parser

def parse_args():
    parser = get_args_parser()
    if 'ipykernel' in sys.modules:
        args, unknown = parser.parse_known_args()
    else:
        args = parser.parse_args()
    return args

class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomEfficientNet, self).__init__()
        self.base_model = models.efficientnet_b3(weights=models.EfficientNet_B3_Weights.IMAGENET1K_V1)
        self.base_model.classifier[1] = nn.Linear(self.base_model.classifier[1].in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

In [8]:
def create_data_loaders(data_path, im_size, batch_size):
    train_dir = os.path.join(data_path, 'train')
    val_dir = os.path.join(data_path, 'val')
    test_dir = os.path.join(data_path, 'test')

    train_transform = transforms.Compose([
        transforms.Resize(im_size),  # Redimensiona las imágenes
        transforms.ToTensor(),       # Convierte imágenes a tensores
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalización
    ])

    val_transform = transforms.Compose([
        transforms.Resize(im_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
    val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)
    test_dataset = datasets.ImageFolder(root=test_dir, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    num_classes = len(train_dataset.classes)
    return train_loader, val_loader, test_loader, num_classes

In [9]:
def validate_model(model, criterion, data_loader, num_classes):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            labels = torch.nn.functional.one_hot(labels, num_classes=num_classes).float()
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.argmax(dim=1).cpu().numpy())

    val_loss = running_loss / len(data_loader)
    val_accuracy = accuracy_score(all_labels, all_preds)
    val_f1 = f1_score(all_labels, all_preds, average='weighted')
    return val_loss, val_accuracy, val_f1, all_labels, all_preds

In [10]:
def train_model(train_loader, val_loader, num_classes, experiment_name, early_stopping_patience, epochs, fold, metrics_file):
    model = CustomEfficientNet(num_classes=num_classes).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Fixed learning rate
    early_stopping = EarlyStopping(patience=early_stopping_patience)

    metrics = []  # To store metrics for each epoch

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            labels = torch.nn.functional.one_hot(labels, num_classes=num_classes).float()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels.argmax(dim=1)).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_loader)
        train_accuracy = correct / total
        val_loss, val_accuracy, val_f1, _, _ = validate_model(model, criterion, val_loader, num_classes)

        print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}, Val F1: {val_f1:.4f}")


        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break

    # Log initial model memory usage
    model_memory_mb = model_memory_usage_alternative(model)
    print(f"Model memory usage: {model_memory_mb:.2f} MB")

    # Save metrics for this epoch
    fold_summary = {
        "fold": fold + 1,
        "train_loss": train_loss,
        "train_accuracy": train_accuracy,
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "val_f1": val_f1,
        "model_memory_mb": model_memory_mb  # Log memory usage
    }
    # Save the metrics to a CSV file
    save_fold_metrics([], fold_summary, metrics_file)

    return model

In [11]:
def test_model(model, test_loader, num_classes):
    criterion = nn.BCEWithLogitsLoss()
    test_loss, test_accuracy, test_f1, all_labels, all_preds = validate_model(model, criterion, test_loader, num_classes)

    print(f"\nTest Metrics:")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test F1 Score (weighted): {test_f1:.4f}")

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds))

In [12]:
def main():
    args = parse_args()
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)

    train_loader, val_loader, test_loader, num_classes = create_data_loaders(args.data_path, args.im_size, args.batch_size)
    kfold = KFold(n_splits=args.k_folds, shuffle=True, random_state=args.seed)

    train_dataset = train_loader.dataset

    os.makedirs(args.experiment_name, exist_ok=True)

    metrics_file = f"{args.experiment_name}/metrics_summary.csv"

    for fold, (train_indices, val_indices) in enumerate(kfold.split(train_dataset)):
        print(f"Fold {fold + 1}/{args.k_folds}")

        train_subset = Subset(train_dataset, train_indices)
        val_subset = Subset(train_dataset, val_indices)

        train_loader = DataLoader(train_subset, batch_size=args.batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=args.batch_size, shuffle=False)

        model = train_model(train_loader, val_loader, num_classes, args.experiment_name, args.early_stopping, args.epochs,fold, metrics_file)
        torch.save(model.state_dict(), f"{args.experiment_name}/fold_{fold + 1}_model_weights.pth")


        print(f"Fold {fold + 1} completed.")

In [13]:
if __name__ == "__main__":
    main()

Fold 1/5


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:08<00:00,  2.83it/s]


Epoch [1/100], Train Loss: 0.4032, Train Acc: 0.5810, Val Loss: 0.2453, Val Acc: 0.7409, Val F1: 0.7315


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:08<00:00,  2.90it/s]


Epoch [2/100], Train Loss: 0.2163, Train Acc: 0.7783, Val Loss: 0.2022, Val Acc: 0.7876, Val F1: 0.7859


...

Epoch [11/100], Train Loss: 0.0232, Train Acc: 0.9830, Val Loss: 0.3315, Val Acc: 0.7875, Val F1: 0.7878


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:08<00:00,  2.97it/s]


Epoch [12/100], Train Loss: 0.0229, Train Acc: 0.9827, Val Loss: 0.3404, Val Acc: 0.7881, Val F1: 0.7878


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:08<00:00,  2.98it/s]


Epoch [13/100], Train Loss: 0.0187, Train Acc: 0.9851, Val Loss: 0.3514, Val Acc: 0.7919, Val F1: 0.7930
Early stopping triggered at epoch 13
Model memory usage: 40.83 MB
Fold 5 completed.
